# Conversion of Raw lidar data to cloud and precipitation classification

## Abstract

This Notebook contains the post processing of the lidar data, including the spatial and temporal integration, the system calibration, background correction, SNR estimation, volume depolarization estimation, Cloud masking and hidrometeor clasification. Quick looks and some statistics are generated as well. Results are saved in netCDF format.

## Import libraries

Generic and own packages stored in "lib" are loaded to be used along the notebook. 

In [2]:
import sys, os
sys.path.append("lib") # adding "lib" path with own packages
from Sigma_mol import sigma_mol # reads the radio sounsing and compute extinction coefficient
from scipy.interpolate import interp1d # to interpolate modeled variables to lidar heights
from lidar_integrating_space_time import Lidar_space_time as lidar_integ #integrates lidar raw data in height and time
from fft_denoising import fft_denoising #maybe not used
from klett import Klett81b #maybe not used
import numpy as np 
import pylab #plots
from DP_simp import DP_simp # Curve simplification
from running_mean import running_mean # runing mean
from time2epoch import time2epoch #maybe not used?
#from cloud_mask_v1 import cloud_mask
from netCDF4 import Dataset 
#from time import sleep
from scipy import stats
from scipy.optimize import curve_fit
from dplots import densplot # make 2D desity plots
from Comb_LidarMRR import Comb_LidarMRR3 as Comb_LidarMRR 
from cloud_mask_v2 import cloud_mask2
from sm_paramTOP import sm_paramTOP
import matplotlib
from copy import copy
from BG_corr import BG_corr
import time
from calendar import timegm

## Load MRR and other parameters

Load MRR data during the period of study. It also configures the font format and color maps

In [8]:
##### output Temporal RESolution
TRES = 10 #min. 

##### output Temporal resolution
VRES = 6 #bins, 1bin = 3.8m # try only 1,2,3,6 bins

#####Load MRR Data
path_MRR = "I:/PHD/Lidar/Inversion_V2/MRR_Data/"
Ze = np.loadtxt(path_MRR + "Ze_"+str(TRES)+"min.txt")
times_MRR = np.loadtxt(path_MRR + "times_"+str(TRES)+"min.txt")
Height_MRR = np.loadtxt(path_MRR + "Height.txt")

Zem = np.ma.masked_where(Ze == -9999, Ze)

#####Load font format

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20}
        
pylab.rc('font', **font)      

#####Color Parameters
cmap = pylab.cm.jet
bounds = np.linspace(1,3,4)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
bounds2 = np.linspace(0,12,14)
norm2 = matplotlib.colors.BoundaryNorm(bounds2, cmap.N)

cmap2 = pylab.cm.get_cmap(cmap.name,8)

## Temporal and vertical integration

In [ ]:

##### Dates
#[ini, end]
year0 = [2017, 2017]
month0 = [9,12]
day0 = [1,31]
t0 = time.time()
##### Routine
for year in range(year0[0],year0[1]+1):
    for month in range(month0[0],month0[1]+1):
        for day in range(day0[0],day0[1]+1):
    
            path_out = "I:/PHD/Lidar/Processing_V3/Signals/"+str(TRES)+"min"+str(VRES)+"bins/"
            filename1 = path_out+"Par90/Par90_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename2 = path_out+"Par10/Par10_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename3 = path_out+"Per/Per_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            
            filename4 = path_out+"Nprofiles/Nprofiles_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min"+".dat"
            
            if os.path.isfile(filename1):
                #Par90 = np.loadtxt(filename1)
                #Par10 = np.loadtxt(filename2)
                #Per   = np.loadtxt(filename3)
                #r   = np.loadtxt(path_out+"R_"+str(TRES)+"min_"+str(VRES)+"bins.dat")
                #npt = np.loadtxt(filename4)
                print str(year)+str(month).zfill(2)+str(day).zfill(2)+" Data loaded"

            else:
                try:
                    mat = lidar_integ(date = str(year)+"."+str(month).zfill(2)+"."+str(day).zfill(2), space = VRES, timee = TRES, 
                              path = "G:/PC_chantal_20190131/MCS6A Data/")
                except:
                    continue
                    #print "what"
                print str(year)+str(month).zfill(2)+str(day).zfill(2)+" Data loaded"
                #sleep(8)
                np.savetxt(filename1,mat[0])
                np.savetxt(filename2,mat[1])
                np.savetxt(filename3,mat[2])
                #np.savetxt(path_out+"R_"+str(TRES)+"min_"+str(VRES)+"bins.dat",mat[3])
                np.savetxt(filename4,mat[4])
                #Par90 = mat[0]
                #Par10 = mat[1]
                #Per = mat[2]
                #npt = mat[4]
print "Elapsed time = ", (time.time() - t0)

## Background correction and SNR

In [15]:
for count in range(1):
    if count > 0: print "Next iteration"
    ##### Dates
    #[ini, end]
    year0 = [2017,2017]
    month0 = [2,12]
    day0 = [1,31]

    ##### Routine
    for year in range(year0[0],year0[1]+1):
        for month in range(month0[0],month0[1]+1):
            for day in range(day0[0],day0[1]+1):

                path_in = "I:/PHD/Lidar/Processing_V3/Signals/"+str(TRES)+"min"+str(VRES)+"bins/"
                filename1 = path_in+"Par90/Par90_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename2 = path_in+"Par10/Par10_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename3 = path_in+"Per/Per_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename4 = path_in+"Nprofiles/Nprofiles_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min"+".dat"

                path_out = "I:/PHD/Lidar/Processing_V3/Signals/"+str(TRES)+"min"+str(VRES)+"bins/"
                
                filename5 = path_in+"Par90_bc/Par90bc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename6 = path_in+"Par10_bc/Par10bc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename7 = path_in+"Per_bc/Perbc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"

                filename8 = path_in+"SNR/SNR_Par90_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename9 = path_in+"SNR/SNR_Par10_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename10 = path_in+"SNR/SNR_Per_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"

                filename11 = path_in+"Background/BG_Par90_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename12 = path_in+"Background/BG_Par10_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                filename13 = path_in+"Background/BG_Per_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
                
                if os.path.isfile(filename13):
                    #print str(year)+str(month).zfill(2)+str(day).zfill(2)+" Data ready"
                    continue
                
                if os.path.isfile(filename1):
                    Par90 = np.loadtxt(filename1)
                    Par10 = np.loadtxt(filename2)
                    Per   = np.loadtxt(filename3)
                    r   = np.loadtxt(path_in+"R_"+str(TRES)+"min_"+str(VRES)+"bins.dat")
                    npt = np.loadtxt(filename4)

                    BG1 = np.zeros(shape = np.shape(Par90)[0])
                    BG10 = np.zeros(shape = np.shape(Par90)[0])
                    BG2 = np.zeros(shape = np.shape(Par90)[0])

                    print str(year)+str(month).zfill(2)+str(day).zfill(2)+" Data loaded"
                else:
                    #print "File not found"
                    continue

                Par_bc = np.zeros(shape = np.shape(Par90))
                Par10_bc = np.zeros(shape = np.shape(Par90))
                Per_bc = np.zeros(shape = np.shape(Per))
                SNR_par = np.zeros(shape = np.shape(Per))
                SNR_par10 = np.zeros(shape = np.shape(Per))
                SNR_per = np.zeros(shape = np.shape(Per))

                for i in range(np.shape(Par90)[0]):
                    try:
                        BG1[i] = BG_corr(Par90[i,:],r[:],year, month, day,rcf0 = 9,pol = 'parallel')[0][1]
                        BG10[i] = BG_corr(Par10[i,:],r[:],year, month, day,rcf0 = 9,pol = 'parallel')[0][1]
                        BG2[i] = BG_corr(Per[i,:],r[:],year, month, day,rcf0 = 9,pol = 'perpendicular')[0][1]  
                    except:
                        continue
                        
                    if BG1[i] < 0: BG1[i] = 0
                    if BG10[i] < 0: BG10[i] = 0
                    if BG2[i] < 0: BG2[i] = 0

                    if BG1[i] > np.nanmean(Par90[i,-50:]): BG1[i] = np.nanmean(Par90[i,-50:])
                    if BG10[i] > np.nanmean(Par10[i,-50:]): BG10[i] = np.nanmean(Par10[i,-50:])
                    if BG2[i] > np.nanmean(Per[i,-50:]): BG2[i] = np.nanmean(Per[i,-50:])

                    Par_bc[i,:] = Par90[i,:]-BG1[i]
                    Par10_bc[i,:] = Par10[i,:]-BG10[i]
                    Per_bc[i,:] = Per[i,:]-BG2[i]  

                    SNR_par[i,:] = (Par_bc[i,:]*(npt[i]*TRES*6)**0.5)/(Par_bc[i,:]+2*(BG1[i]))**0.5
                    SNR_par10[i,:] = (Par10_bc[i,:]*(npt[i]*TRES*6)**0.5)/(Par10_bc[i,:]+2*(BG10[i]))**0.5
                    SNR_per[i,:] = (Per_bc[i,:]*(npt[i]*TRES*6)**0.5)/(Per_bc[i,:]+2*(BG2[i]))**0.5

                    Par_bc[i,:] = (Par_bc[i,:])*r**2
                    Par10_bc[i,:] = (Par10_bc[i,:])*r**2
                    Per_bc[i,:] = (Per_bc[i,:])*r**2  

                np.savetxt(filename5,Par_bc)
                np.savetxt(filename6,Par10_bc)
                np.savetxt(filename7,Per_bc)

                np.savetxt(filename8,SNR_par)
                np.savetxt(filename9,SNR_par10)
                np.savetxt(filename10,SNR_per)

                np.savetxt(filename11,BG1)
                np.savetxt(filename12,BG10)
                np.savetxt(filename13,BG2)
    #print "waiting 15 minutes..."
    #time.sleep(15*60)
    #print "iteration = ", count
    #month00 = month

## Calibration system 

In [22]:
## load Calibration parameters
txt = open("I:/PHD/Lidar/Processing_V3/Calibration_sys/CalSys_"+str(TRES)+"min_"+str(VRES)+"bins_1.csv","r")
CalSys_header = txt.readline()

CalSys_time = []
CalSys_Cpar = []
CalSys_Cper = []
CalSys_K = []

for l in txt.readlines():
    CalSys_time.append(float(l.split(",")[1]))
    CalSys_Cpar.append(float(l.split(",")[3]))
    CalSys_Cper.append(float(l.split(",")[4]))  
    CalSys_K.append(float(l.split(",")[5]))  
    
CalSys_time = np.array(CalSys_time)
CalSys_Cpar = np.array(CalSys_Cpar)
CalSys_Cper = np.array(CalSys_Cper)
CalSys_K    = np.array(CalSys_K)

# overlapping function
path_in = "I:/PHD/Lidar/Processing_V3/Signals/"+str(TRES)+"min"+str(VRES)+"bins/"

#O = np.loadtxt(path_in + "Overlapping_"+str(TRES)+"min_"+str(VRES)+"bins.dat")
#Or = np.loadtxt(path_in + "Overlapping_"+str(TRES)+"min_"+str(VRES)+"bins_R.dat")
O = np.loadtxt(path_in+"Overlapping_"+str(TRES)+"min_"+str(VRES)+"bins_08032017_14UTC2.dat")
Or = np.loadtxt(path_in+"Overlapping_"+str(TRES)+"min_"+str(VRES)+"bins_08032017_14UTC_r2.dat")
##### Signal, X and Depolarization calibration
year0 = [2017,2017]
month0 = [2,8]
day0 = [1,31]

for year in range(year0[0],year0[1]+1):
    for month in range(month0[0],month0[1]+1):
        for day in range(day0[0],day0[1]+1):

            filename1 = path_in+"Par90_bc/Par90bc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename2 = path_in+"Per_bc/Perbc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            path_out = "I:/PHD/Lidar/Processing_V3/SysCalibrated/"+str(TRES)+"min"+str(VRES)+"bins/"
            
            if (os.path.isdir(path_out+"Bpar") == False): os.makedirs(path_out+"Bpar")
            if (os.path.isdir(path_out+"Bper") == False): os.makedirs(path_out+"Bper")
            if (os.path.isdir(path_out+"DepRatio") == False): os.makedirs(path_out+"DepRatio")
            if (os.path.isdir(path_out+"X") == False): os.makedirs(path_out+"X")

            filename3 = path_out+"Bpar/Bpar_sc_O_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename4 = path_out+"Bper/Bper_sc_O_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename5 = path_out+"DepRatio/DepRatio_sc_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
            filename6 = path_out+"X/X_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"

            if os.path.isfile(filename6):
                continue
            if os.path.isfile(filename1):
                Par = np.loadtxt(filename1)
                Per = np.loadtxt(filename2)
                r   = np.loadtxt(path_in+"R_"+str(TRES)+"min_"+str(VRES)+"bins.dat")
                
                utc_time = time.strptime(str(year)+"-"+str(month).zfill(2)+"-"+str(day).zfill(2)+" "+"00:00:00", "%Y-%m-%d %H:%M:%S")
                epoch_time = timegm(utc_time)
                
                pix_cal = np.squeeze(np.where(abs(CalSys_time-epoch_time) == np.min(abs(CalSys_time-epoch_time))))
                
                Cpar = np.mean(CalSys_Cpar[pix_cal])
                Cper = np.mean(CalSys_Cper[pix_cal])
                
                #Attenuated Backscatter coefficient
                Bpar = Par/Cpar
                Bper = Per/Cper
                
                #Linear Depolarization Ratio
                DepRatio = (Bper/Bpar)
                
                #Overlapping Correction
                
                Bpar_O = np.zeros(shape = np.shape(Par[:,int(11*3./VRES):]))
                Bper_O = np.zeros(shape = np.shape(Per[:,int(11*3./VRES):]))
                
                for j in range(np.shape(Par)[0]):
                    Bpar_O[j,:] = Bpar[j,int(11*3./VRES):]*O 
                    Bper_O[j,:] = Bper[j,int(11*3./VRES):]*O 
                
                #Attenuation between consecutive layers: X
                X = np.zeros(shape = np.shape(Bpar_O))

                for i in range(15,np.size(Or)-2):
                    X[:,i] = np.log10(Bpar_O[:,i]/Bpar_O[:,i+1])
                
                #save Data
                np.savetxt(filename3,Bpar_O)
                np.savetxt(filename4,Bper_O)
                np.savetxt(filename5,DepRatio)
                np.savetxt(filename6,X)
                
                

C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:86: RuntimeWarning: invalid value encountered in log10
C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:71: RuntimeWarning: divide by zero encountered in divide
C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:86: RuntimeWarning: divide by zero encountered in divide
C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:86: RuntimeWarning: divide by zero encountered in log10
C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:86: RuntimeWarning: invalid value encountered in divide
C:\Users\duran\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:71: RuntimeWarning: invalid value encountered in divide


## Cloud Masking

In [ ]:
##### 
year0 = [2017,2017]
month0 = [2,2]
day0 = [1,1]

nstd = 3
sm= 5
dPlot = False
check = False
slr = 0
th = 1.1


path_in = "I:/PHD/Lidar/Processing_V3/SysCalibrated/"+str(TRES)+"min"+str(VRES)+"bins/"

r = np.loadtxt(path_in + "Overlapping_"+str(TRES)+"min_"+str(VRES)+"bins_R.dat")

for year in range(year0[0],year0[1]+1):
    for month in range(month0[0],month0[1]+1):
        for day in range(day0[0],day0[1]+1):

        filename1 = path_in+"Bpar/Bpar_sc_O_"+str(year)+str(month).zfill(2)+str(day).zfill(2)+"_"+str(TRES)+"min_"+str(VRES)+"bins.dat"
  
        if os.path.isfile(filename6):
            continue
            
        if os.path.isfile(filename1):
        
            Par = np.loadtxt(filename1)
            BASES = np.zeros(shape = (np.size(Par[:,0]),20))
            TOPS = np.zeros(shape = (np.size(Par[:,0]),20))
            Data_mask = np.zeros(shape = np.shape(Par))
            
            Y2 = Par#Y2[15:]
            R = r#_[3:][15:]
            AA=running_mean(Y2,sm)#[sm-1:]
            rr=R[(sm-1)/2:-(sm-1)/2]#pd.rolling_mean(r,sm)[sm-1:]

            pix=np.squeeze(np.where(AA <0))

            AAA = np.zeros(shape = np.size(AA)-1)
            sum = 0
            for i0 in np.linspace(np.size(AA)-2,0,np.size(AA)-1):
                i = int(i0)
                #print i,AA[i],AA[i+1]
                if ((AA[i]>0) & (AA[i+1]>0)):
                    sum = sum+1
                    #print sum
                    AAA[i]=sum
                else:
                    sum = 0
                    AAA[i]=sum    

            AAA[1:] = AAA[1:]-AAA[0:-1]

            pix2 = np.squeeze(np.where(AAA > 20))

            if np.size(pix2)>1:

                deltaH = 0
                for nlayer in range(np.size(pix2)):
                    #print "nlayer",nlayer

                    rrr = rr[pix2[nlayer]:pix2[nlayer]+int(AAA[pix2[nlayer]])+1]

                    pix3=np.squeeze(np.where((R[:-1] >= rrr[0]) & (R[:-1] <rrr[-1])))#r_[3:-1]

                    Y3 = Y2[pix3[0]-(sm-1)/2:pix3[-1]+(sm-1)/2+2]
                    R3 = R[pix3[0]-(sm-1)/2:pix3[-1]+(sm-1)/2+2]
                    #stop
                    c_mask, baseH, topH, layers, layers_mask = cloud_mask2(Y3,R3,doPlot = dPlot, th = th,nstd=nstd, sm = sm, check = check)
                    #pylab.yscale("log")
                    #stop
                    Data_mask[int(h),np.squeeze(np.where(r_ == np.nanmin(R3))):np.squeeze(np.where(r_ == np.nanmax(R3)))+1] = 1.
                    if np.sum(baseH) != -9999:
                        #print baseH,topH
                        BASES5[int(h),0+deltaH:np.size(baseH)+deltaH] = baseH
                        TOPS5[int(h),0:np.size(topH)+deltaH] = topH
                        deltaH = np.size(baseH)

            elif np.size(pix2)>=1:

                rrr = rr[pix2:pix2+int(AAA[pix2])+1]

                pix3=np.squeeze(np.where((R[:-1] >= rrr[0]) & (R[:-1] <rrr[-1]))) #r_[3:-1]

                Y3 = Y2[pix3[0]-(sm-1)/2:pix3[-1]+(sm-1)/2+2]
                R3 = R[pix3[0]-(sm-1)/2:pix3[-1]+(sm-1)/2+2] #r_

                Data_mask[int(h),np.squeeze(np.where(r_ == np.nanmin(R3))):np.squeeze(np.where(r_ == np.nanmax(R3)))+1] = 1.

                #fig, ax1 = pylab.subplots(figsize = (8,5))  
                #pylab.xticks(range(11))
                #pylab.axis([0,10,1,1e6])

                #pylab.plot(r_[3:-1],Y2,color = "gray")

                c_mask, baseH, topH, layers, layers_mask = cloud_mask2(Y3,R3,doPlot = dPlot, th = th,nstd=nstd, sm = sm, check = check)
            #pylab.show()
                #stop
                if np.sum(baseH) != -9999:
                    BASES5[int(h),0:np.size(baseH)] = baseH
                    TOPS5[int(h),0:np.size(topH)] = topH            
